In [1]:
# !pip install gapminder
# !pip install bokeh

In [2]:
import pandas as pd
from bokeh.io import curdoc
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider, Select, Button
from bokeh.layouts import column, row
from bokeh.palettes import Spectral6

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/abrahamzetz/gapminder_bokeh/main/gapminder_tidy.csv", index_col='Year')#gapminder.gapminder
data.head()

,Country,fertility,life,population,child_mortality,gdp,region
Year,,,,,,,
1964,Afghanistan,7.671,33.639,10474903.0,339.7,1182.0,South Asia
1965,Afghanistan,7.671,34.152,10697983.0,334.1,1182.0,South Asia
1966,Afghanistan,7.671,34.662,10927724.0,328.7,1168.0,South Asia
1967,Afghanistan,7.671,35.170,11163656.0,323.3,1173.0,South Asia
1968,Afghanistan,7.671,35.674,11411022.0,318.1,1187.0,South Asia


In [4]:
# Make the base plot

# Define initial plot axes
x = 'fertility'
xmin, xmax = min(data[x]), max(data[x])

y = 'life'
ymin, ymax = min(data[y]), max(data[y])

year = min(data.index)


# Define labels to make axis labels updated
label= {
    'fertility' : 'Fertilidad (# hijos por mujer)',
    'life' : 'Esperanza de vida (años)',
    'child_mortality' : 'Tasa de mortalidad infantil',
    'gdp': 'PIB país'
}

diccionario = {
    'x'       : data.loc[year, x],
    'y'       : data.loc[year, y],
    'country' : data.loc[year, 'Country'],
    'pop'     : data.loc[year, 'population'],
    'pop_size': data.loc[year, 'population'] / 20000000 + 2,
    'region'  : data.loc[year, 'region']
}


# de donde salen los datos
source = ColumnDataSource(data = diccionario)

# Mapeamos los colores
regions_list = data['region'].unique().tolist()
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)


# Creamos la figura
plot = figure(title='Gráfico de {} vs {} para el año {}'.format(label[y], label[x], year), height=700, width=1000,
              x_range=(xmin, xmax), y_range=(ymin, ymax))

plot.circle(x='x', y='y', fill_alpha=0.6, source=source,
            color=dict(field='region', transform=color_mapper), size='pop_size', legend_group='region')

plot.legend.location = 'top_right'

plot.xaxis.axis_label = label[x]

plot.yaxis.axis_label = label[y]

In [5]:
# Creamos un HoverTool
hover = HoverTool(tooltips=[
    ('País', '@country'),
    ('Región', '@region'),
    ('Población', '@pop{0,0}'),
    (label[x], '@x{0,0.00}'),
    (label[y], '@y{0,0.00}')
])

plot.add_tools(hover)

In [6]:
# Define the callback function for interacting with the widgets
def update_plot(attr, old, new):
    #Use global hover to update HoverTool    
    global hover
    
    #Get value of current widgets
    year = slider.value
    x = x_select.value
    y = y_select.value
    
    # Label plot axes
    plot.xaxis.axis_label = label[x]
    plot.yaxis.axis_label = label[y]
    
    # Set new_data
    new_data = {
        'x'       : data.loc[year, x],
        'y'       : data.loc[year, y],
        'country' : data.loc[year, 'Country'],
        'pop'     : data.loc[year, 'population'],
        'pop_size': data.loc[year, 'population'] / 20000000 + 2,
        'region'  : data.loc[year, 'region']
    }
    
    # Assign new_data to source.data
    source.data = new_data

    # Set the range of all axes
    plot.x_range.start = min(data[x])
    plot.x_range.end = max(data[x])
    plot.y_range.start = min(data[y])
    plot.y_range.end = max(data[y])

    # Add title to plot
    plot.title.text = 'Gráfico de {} vs {} para el año {}'.format(label[y], label[x], year)
    
    # Updating the hover tools
    plot.tools.remove(hover)
    
    hover = HoverTool(tooltips=[
        ('País', '@country'),
        ('Región', '@region'),
        ('Población', '@pop{0,0}'),
        (label[x], '@x{0,0.00}'),
        (label[y], '@y{0,0.00}')
    ])

    plot.add_tools(hover)

In [7]:
# Create a year slider
slider = Slider(start=min(data.index), end=max(data.index), step=1, value=min(data.index), title='Year')

# Attach the callback to slider
slider.on_change('value', update_plot)

# Create the list of the dropdown menus
dropdown = ['fertility', 'life', 'child_mortality', 'gdp']

# Create a dropdown Select widget for the x data
x_select = Select(
    options=dropdown,
    value=dropdown[0],
    title='eje x'
)

# Attach the update_plot callback to x dropdown
x_select.on_change('value', update_plot)

# Create a dropdown Select widget for the y data
y_select = Select(
    options=dropdown,
    value=dropdown[1],
    title='eje y'
)

# Attach the update_plot callback to y dropdown
y_select.on_change('value', update_plot)

In [15]:
# Create play-pause button

#define the callback function to animate the slider when clicking the button
def animate_update():
    year = slider.value + 1
    if year > max(data.index):
        year = min(data.index)
    slider.value = year

#create global variable for callback
callback_animate = None

#define the callback function for clicking the button
def animate():
    global callback_animate
    if button.label == '► Play':
        button.label = '❚❚ Pause'
        callback_animate = curdoc().add_periodic_callback(animate_update, 200)
    else:
        button.label = '► Play'
        curdoc().remove_periodic_callback(callback_animate)

#create the button and set on-click callback
button = Button(label='► Play', width=60)
button.on_click(animate)

In [9]:
# Create layout and add to current document
layout = row(column(slider, button, x_select, y_select), plot)
curdoc().add_root(layout)
curdoc().title = 'Gapminder'

In [10]:
#!bokeh serve --show GapBOKEH.ipynb
!bokeh serve --show --port 4004 GapBOKEH.ipynb

^C


In [16]:
from bokeh.io import output_notebook
from bokeh.io import output_file
output_file('gapminder.html')
# from bokeh.io import output_notebook
# output_notebook(plot)

In [18]:
from bokeh.plotting import figure, output_file, save

output_file(filename="custom_filename.html", title="Static HTML file")
#save(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



'C:\\Users\\rosme\\Registraduria\\Borradores_dashboards\\gapminder\\custom_filename.html'